In [106]:
import pandas as pd
import numpy as np
import sklearn
import sklearn.preprocessing as PP
import sys
from sklearn.model_selection import train_test_split #version 0.18
from sklearn import svm
from sklearn.model_selection import KFold #version 0.18
from sklearn.metrics import accuracy_score
from tqdm import tqdm
from sklearn import linear_model
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier

In [3]:
data = pd.read_csv('/Users/JJ/Documents/SIPA/Semester 4/Artificial Intelligence/assignment3/input3.csv')#, header=None, names=['age', 'weight', 'height'])
#data

In [4]:
# sys_input = sys.argv
# data_path = sys_input[1]
# output_path = sys_input[2]
# data = pd.read_csv(data_path, header=None, names=['feature_1', 'feature_2', 'label'])

In [5]:
data_split = train_test_split(data, test_size=0.4, stratify=data['label'])

In [6]:
data_train = data_split[0]
data_test = data_split[1]

In [7]:
points = []
labels = []

for r in range(len(data_train)):
    points.append([data_train.iloc[r,0],data_train.iloc[r,1]])
    labels.append(data_train.iloc[r,2])

In [8]:
points_test = []
labels_test = []

for r in range(len(data_test)):
    points_test.append([data_test.iloc[r,0],data_test.iloc[r,1]])
    labels_test.append(data_test.iloc[r,2])

In [9]:
def run_model(s_kernel, training, training_labels, s_c=1, s_degree=3, s_gamma='auto', validation_training=None, validation_labels=None):
    clf = svm.SVC(kernel=s_kernel, C=s_c, degree=s_degree, gamma=s_gamma)
    clf.fit(training, training_labels)
    return accuracy_score(validation_labels, clf.predict(validation_training))

In [56]:
def run_model_logistic(training, training_labels, s_c=1, s_degree=3, s_gamma='auto', validation_training=None, validation_labels=None):
    clf = linear_model.LogisticRegression(C=s_c)
    clf.fit(training, training_labels)
    return accuracy_score(validation_labels, clf.predict(validation_training))

In [69]:
def run_model_kneighbors(training, training_labels, s_neighbors=1, s_leaf=3, validation_training=None, validation_labels=None):
    clf = KNeighborsClassifier(n_neighbors=s_neighbors, leaf_size=s_leaf)
    clf.fit(training, training_labels)
    return accuracy_score(validation_labels, clf.predict(validation_training))

In [107]:
def run_model_dtree(training, training_labels, s_depth, s_split, validation_training=None, validation_labels=None):
    clf = tree.DecisionTreeClassifier(max_depth=s_depth, min_samples_split=s_split)
    clf.fit(training, training_labels)
    return accuracy_score(validation_labels, clf.predict(validation_training))

In [110]:
def run_model_rforrest(training, training_labels, s_depth, s_split, validation_training=None, validation_labels=None):
    clf = RandomForestClassifier(max_depth=s_depth, min_samples_split=s_split)
    clf.fit(training, training_labels)
    return accuracy_score(validation_labels, clf.predict(validation_training))

In [11]:
splits_to_do = 5
kf = KFold(n_splits=splits_to_do)

X = np.array(points)
y = np.array(labels)

c_values = [0.1, 0.5, 1, 5, 10, 50, 100]

accuracies = []
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    for i in c_values:
        temp_score = run_model('linear', s_c=i, training=X_train, training_labels=y_train, validation_training=X_test, validation_labels=y_test)
        accuracies.append([i,temp_score])

In [12]:
final_accuracy = []

for i in c_values:
    total = 0
    count = 0
    for a in accuracies:
        if a[0] == i:
            total = total + a[1]
            count += 1
            if count == splits_to_do:
                total = total / splits_to_do
                final_accuracy.append([i, total])

In [13]:
top_accuracy = []

for f in range(len(final_accuracy)):
    top_accuracy.append(final_accuracy[f][1])

In [14]:
top_accuracy = max(top_accuracy)

In [15]:
report = []

report.append(['linear',top_accuracy,run_model('linear', training=points, training_labels=labels, validation_training=points_test, validation_labels=labels_test)])

In [16]:
report

[['linear', 0.59000000000000008, 0.58999999999999997]]

In [18]:
splits_to_do = 5
kf = KFold(n_splits=splits_to_do)

X = np.array(points)
y = np.array(labels)

c_values = [0.1, 1, 3]
degree_values = [4, 5, 6]
gamma_values = [0.1, 1]

accuracies = []
for train_index, test_index in tqdm(kf.split(X)):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    for i in tqdm(c_values):
        for d in tqdm(degree_values):
            for g in tqdm(gamma_values):
                temp_score = run_model('poly', s_c=i, s_degree=d, s_gamma=g, training=X_train, training_labels=y_train, validation_training=X_test, validation_labels=y_test)
                accuracies.append([i,d,g,temp_score])

0it [00:00, ?it/s]
  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]


  0%|          | 0/2 [00:00<?, ?it/s]


100%|██████████| 2/2 [00:00<00:00, 14.96it/s]




 33%|███▎      | 1/3 [00:00<00:00,  7.32it/s]


  0%|          | 0/2 [00:00<?, ?it/s]


100%|██████████| 2/2 [00:01<00:00,  1.37it/s]




 67%|██████▋   | 2/3 [00:01<00:00,  1.87it/s]


  0%|          | 0/2 [00:00<?, ?it/s]


100%|██████████| 2/2 [00:10<00:00,  5.39s/it]




100%|██████████| 3/3 [00:12<00:00,  3.61s/it]


 33%|███▎      | 1/3 [00:12<00:24, 12.40s/it]

  0%|          | 0/3 [00:00<?, ?it/s]


  0%|          | 0/2 [00:00<?, ?it/s]


100%|██████████| 2/2 [00:00<00:00,  2.53it/s]




 33%|███▎      | 1/3 [00:00<00:01,  1.26it/s]


  0%|          | 0/2 [00:00<?, ?it/s]


100%|██████████| 2/2 [00:16<00:00,  8.47s/it]




 67%|██████▋   | 2/3 [00:17<00:05,  5.64s/it]


  0%|          | 0/2 [00:00<?, ?it/s]


100%|██████████| 2/2 [00:52<00:00, 26.23s/it]




100%|██████████| 3/3 [01:10<00:00, 

In [19]:
accuracies

[[0.1, 4, 0.1, 0.58333333333333337],
 [0.1, 4, 1, 0.69999999999999996],
 [0.1, 5, 0.1, 0.58333333333333337],
 [0.1, 5, 1, 0.71666666666666667],
 [0.1, 6, 0.1, 0.58333333333333337],
 [0.1, 6, 1, 0.68333333333333335],
 [1, 4, 0.1, 0.58333333333333337],
 [1, 4, 1, 0.71666666666666667],
 [1, 5, 0.1, 0.58333333333333337],
 [1, 5, 1, 0.69999999999999996],
 [1, 6, 0.1, 0.6333333333333333],
 [1, 6, 1, 0.65000000000000002],
 [3, 4, 0.1, 0.58333333333333337],
 [3, 4, 1, 0.68333333333333335],
 [3, 5, 0.1, 0.58333333333333337],
 [3, 5, 1, 0.68333333333333335],
 [3, 6, 0.1, 0.68333333333333335],
 [3, 6, 1, 0.6166666666666667],
 [0.1, 4, 0.1, 0.59999999999999998],
 [0.1, 4, 1, 0.6333333333333333],
 [0.1, 5, 0.1, 0.59999999999999998],
 [0.1, 5, 1, 0.66666666666666663],
 [0.1, 6, 0.1, 0.59999999999999998],
 [0.1, 6, 1, 0.68333333333333335],
 [1, 4, 0.1, 0.59999999999999998],
 [1, 4, 1, 0.66666666666666663],
 [1, 5, 0.1, 0.59999999999999998],
 [1, 5, 1, 0.65000000000000002],
 [1, 6, 0.1, 0.616666666666

In [24]:
final_accuracy_poly = []

for i in c_values:
    for d in degree_values:
        for g in gamma_values:
            count = 0
            total = 0
            for a in accuracies:
                if a[0] == i and a[1] == d and a[2] == g:
                    total = total + a[3]
                    count += 1
                    if count == splits_to_do:
                        total = total / splits_to_do
                        final_accuracy_poly.append([i, d, g, total])

In [25]:
final_accuracy_poly

[[0.1, 4, 0.1, 0.59000000000000008],
 [0.1, 4, 1, 0.64000000000000001],
 [0.1, 5, 0.1, 0.59000000000000008],
 [0.1, 5, 1, 0.66333333333333333],
 [0.1, 6, 0.1, 0.59000000000000008],
 [0.1, 6, 1, 0.66666666666666663],
 [1, 4, 0.1, 0.56333333333333335],
 [1, 4, 1, 0.64666666666666672],
 [1, 5, 0.1, 0.59000000000000008],
 [1, 5, 1, 0.65333333333333332],
 [1, 6, 0.1, 0.59333333333333338],
 [1, 6, 1, 0.66333333333333333],
 [3, 4, 0.1, 0.57000000000000006],
 [3, 4, 1, 0.64000000000000001],
 [3, 5, 0.1, 0.58666666666666667],
 [3, 5, 1, 0.65000000000000002],
 [3, 6, 0.1, 0.6333333333333333],
 [3, 6, 1, 0.63333333333333341]]

In [26]:
top_accuracy_poly = []

for f in range(len(final_accuracy_poly)):
    top_accuracy_poly.append(final_accuracy_poly[f][3])

In [31]:
top_accuracy_poly = max(top_accuracy_poly)

In [32]:
report.append(['poly',top_accuracy_poly,run_model('poly', training=points, training_labels=labels, validation_training=points_test, validation_labels=labels_test, s_c=0.1, s_degree=6, s_gamma=1)])

In [33]:
report

[['linear', 0.59000000000000008, 0.58999999999999997],
 ['poly',
  [0.59000000000000008,
   0.64000000000000001,
   0.59000000000000008,
   0.66333333333333333,
   0.59000000000000008,
   0.66666666666666663,
   0.56333333333333335,
   0.64666666666666672,
   0.59000000000000008,
   0.65333333333333332,
   0.59333333333333338,
   0.66333333333333333,
   0.57000000000000006,
   0.64000000000000001,
   0.58666666666666667,
   0.65000000000000002,
   0.6333333333333333,
   0.63333333333333341],
  0.67500000000000004],
 ['poly', 0.66666666666666663, 0.67500000000000004]]

In [38]:
report

[['linear', 0.59000000000000008, 0.58999999999999997],
 ['poly', 0.66666666666666663, 0.67500000000000004]]

In [39]:
splits_to_do = 5
kf = KFold(n_splits=splits_to_do)

X = np.array(points)
y = np.array(labels)

c_values = [0.1, 0.5, 1, 5, 10, 50, 100]
gamma_values = [0.1, 0.5, 1, 3, 6, 10]

accuracies = []
for train_index, test_index in tqdm(kf.split(X)):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    for i in tqdm(c_values):
        for g in tqdm(gamma_values):
            temp_score = run_model('rbf', s_c=i, s_gamma=g, training=X_train, training_labels=y_train, validation_training=X_test, validation_labels=y_test)
            accuracies.append([i,g,temp_score])

0it [00:00, ?it/s]
  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

100%|██████████| 6/6 [00:00<00:00, 81.61it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

 83%|████████▎ | 5/6 [00:00<00:00, 49.42it/s]

 29%|██▊       | 2/7 [00:00<00:00,  9.56it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

100%|██████████| 6/6 [00:00<00:00, 194.18it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

100%|██████████| 6/6 [00:00<00:00, 186.82it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

 71%|███████▏  | 5/7 [00:00<00:00, 11.48it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

100%|██████████| 6/6 [00:00<00:00, 182.63it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

100%|██████████| 6/6 [00:00<00:00, 113.00it/s]
1it [00:00,  2.15it/s]00:00<00:00, 15.78it/s]
  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

100%|██████████| 6/6 [00:00<00:00, 113.61it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

 29%|██▊       | 2/7 [00:00<00:00, 14.99it/s]

  0%|          | 0/6 [00:00<?, ?

In [40]:
accuracies

[[0.1, 0.1, 0.58333333333333337],
 [0.1, 0.5, 0.58333333333333337],
 [0.1, 1, 0.68333333333333335],
 [0.1, 3, 0.8666666666666667],
 [0.1, 6, 0.66666666666666663],
 [0.1, 10, 0.58333333333333337],
 [0.5, 0.1, 0.58333333333333337],
 [0.5, 0.5, 0.91666666666666663],
 [0.5, 1, 0.94999999999999996],
 [0.5, 3, 0.91666666666666663],
 [0.5, 6, 0.93333333333333335],
 [0.5, 10, 0.91666666666666663],
 [1, 0.1, 0.58333333333333337],
 [1, 0.5, 0.91666666666666663],
 [1, 1, 0.93333333333333335],
 [1, 3, 0.90000000000000002],
 [1, 6, 0.91666666666666663],
 [1, 10, 0.93333333333333335],
 [5, 0.1, 0.69999999999999996],
 [5, 0.5, 0.91666666666666663],
 [5, 1, 0.94999999999999996],
 [5, 3, 0.94999999999999996],
 [5, 6, 0.93333333333333335],
 [5, 10, 0.93333333333333335],
 [10, 0.1, 0.69999999999999996],
 [10, 0.5, 0.8833333333333333],
 [10, 1, 0.98333333333333328],
 [10, 3, 0.96666666666666667],
 [10, 6, 0.91666666666666663],
 [10, 10, 0.93333333333333335],
 [50, 0.1, 0.83333333333333337],
 [50, 0.5, 0.9

In [41]:
final_accuracy_poly = []

for i in c_values:
    for g in gamma_values:
        count = 0
        total = 0
        for a in accuracies:
            if a[0] == i and a[1] == g:
                total = total + a[2]
                count += 1
                if count == splits_to_do:
                    total = total / splits_to_do
                    final_accuracy_poly.append([i, g, total])

In [42]:
final_accuracy_poly

[[0.1, 0.1, 0.59000000000000008],
 [0.1, 0.5, 0.59000000000000008],
 [0.1, 1, 0.70999999999999996],
 [0.1, 3, 0.80333333333333334],
 [0.1, 6, 0.69000000000000006],
 [0.1, 10, 0.59333333333333338],
 [0.5, 0.1, 0.56333333333333335],
 [0.5, 0.5, 0.82333333333333347],
 [0.5, 1, 0.90999999999999992],
 [0.5, 3, 0.90000000000000002],
 [0.5, 6, 0.92666666666666653],
 [0.5, 10, 0.92666666666666653],
 [1, 0.1, 0.57666666666666666],
 [1, 0.5, 0.87999999999999989],
 [1, 1, 0.91999999999999993],
 [1, 3, 0.90999999999999992],
 [1, 6, 0.92999999999999994],
 [1, 10, 0.92666666666666653],
 [5, 0.1, 0.66000000000000003],
 [5, 0.5, 0.90666666666666662],
 [5, 1, 0.92333333333333323],
 [5, 3, 0.91999999999999993],
 [5, 6, 0.91999999999999993],
 [5, 10, 0.91999999999999993],
 [10, 0.1, 0.66000000000000003],
 [10, 0.5, 0.90333333333333332],
 [10, 1, 0.93333333333333324],
 [10, 3, 0.93333333333333324],
 [10, 6, 0.91666666666666663],
 [10, 10, 0.92333333333333323],
 [50, 0.1, 0.77666666666666673],
 [50, 0.5, 0

In [44]:
top_accuracy_poly = []

for f in range(len(final_accuracy_poly)):
    top_accuracy_poly.append(final_accuracy_poly[f][2])

In [45]:
top_accuracy_poly

[0.59000000000000008,
 0.59000000000000008,
 0.70999999999999996,
 0.80333333333333334,
 0.69000000000000006,
 0.59333333333333338,
 0.56333333333333335,
 0.82333333333333347,
 0.90999999999999992,
 0.90000000000000002,
 0.92666666666666653,
 0.92666666666666653,
 0.57666666666666666,
 0.87999999999999989,
 0.91999999999999993,
 0.90999999999999992,
 0.92999999999999994,
 0.92666666666666653,
 0.66000000000000003,
 0.90666666666666662,
 0.92333333333333323,
 0.91999999999999993,
 0.91999999999999993,
 0.91999999999999993,
 0.66000000000000003,
 0.90333333333333332,
 0.93333333333333324,
 0.93333333333333324,
 0.91666666666666663,
 0.92333333333333323,
 0.77666666666666673,
 0.92999999999999994,
 0.93333333333333335,
 0.92666666666666653,
 0.92333333333333323,
 0.92999999999999994,
 0.80666666666666664,
 0.94333333333333336,
 0.93333333333333324,
 0.93666666666666654,
 0.92333333333333323,
 0.92999999999999994]

In [46]:
top_accuracy_poly = max(top_accuracy_poly)

In [47]:
top_accuracy_poly

0.94333333333333336

In [48]:
report.append(['rbf',top_accuracy_poly,run_model('rbf', training=points, training_labels=labels, validation_training=points_test, validation_labels=labels_test, s_c=100, s_gamma=.5)])

In [49]:
report

[['linear', 0.59000000000000008, 0.58999999999999997],
 ['poly', 0.66666666666666663, 0.67500000000000004],
 ['rbf', 0.94333333333333336, 0.95999999999999996]]

In [62]:
splits_to_do = 5
kf = KFold(n_splits=splits_to_do)

X = np.array(points)
y = np.array(labels)

c_values = [0.1, 0.5, 1, 5, 10, 50, 100]

accuracies = []
for train_index, test_index in tqdm(kf.split(X)):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    for i in tqdm(c_values):
        temp_score = run_model_logistic(s_c=i, training=X_train, training_labels=y_train, validation_training=X_test, validation_labels=y_test)
        accuracies.append([i,temp_score])

0it [00:00, ?it/s]
  0%|          | 0/7 [00:00<?, ?it/s]
5it [00:00, 93.45it/s]00:00<00:00, 732.14it/s]


In [63]:
accuracies

[[0.1, 0.58333333333333337],
 [0.5, 0.58333333333333337],
 [1, 0.58333333333333337],
 [5, 0.58333333333333337],
 [10, 0.58333333333333337],
 [50, 0.58333333333333337],
 [100, 0.58333333333333337],
 [0.1, 0.59999999999999998],
 [0.5, 0.59999999999999998],
 [1, 0.59999999999999998],
 [5, 0.59999999999999998],
 [10, 0.59999999999999998],
 [50, 0.59999999999999998],
 [100, 0.59999999999999998],
 [0.1, 0.6166666666666667],
 [0.5, 0.6166666666666667],
 [1, 0.6166666666666667],
 [5, 0.6166666666666667],
 [10, 0.6166666666666667],
 [50, 0.6166666666666667],
 [100, 0.6166666666666667],
 [0.1, 0.66666666666666663],
 [0.5, 0.66666666666666663],
 [1, 0.66666666666666663],
 [5, 0.66666666666666663],
 [10, 0.66666666666666663],
 [50, 0.66666666666666663],
 [100, 0.66666666666666663],
 [0.1, 0.48333333333333334],
 [0.5, 0.48333333333333334],
 [1, 0.48333333333333334],
 [5, 0.48333333333333334],
 [10, 0.48333333333333334],
 [50, 0.48333333333333334],
 [100, 0.48333333333333334]]

In [64]:
final_accuracy_logistic = []

for i in c_values:
    count = 0
    total = 0
    for a in accuracies:
        if a[0] == i:
            total = total + a[1]
            count += 1
            if count == splits_to_do:
                total = total / splits_to_do
                final_accuracy_logistic.append([i, total])

In [65]:
final_accuracy_logistic

[[0.1, 0.59000000000000008],
 [0.5, 0.59000000000000008],
 [1, 0.59000000000000008],
 [5, 0.59000000000000008],
 [10, 0.59000000000000008],
 [50, 0.59000000000000008],
 [100, 0.59000000000000008]]

In [66]:
report.append(['logistic',final_accuracy_logistic[0][1],run_model_logistic(training=points, training_labels=labels, validation_training=points_test, validation_labels=labels_test, s_c=100)])

In [77]:
report

[['linear', 0.59000000000000008, 0.58999999999999997],
 ['poly', 0.66666666666666663, 0.67500000000000004],
 ['rbf', 0.94333333333333336, 0.95999999999999996],
 ['logistic', 0.59000000000000008, 0.58999999999999997]]

In [82]:
neigh_values = list(range(50))
leaf_values = list(range(12))
for l in range(len(leaf_values)):
    leaf_values[l] = (leaf_values[l]+1)*5

In [84]:
accuracies = []
for train_index, test_index in tqdm(kf.split(X)):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    for i in tqdm(neigh_values):
        for l in leaf_values:
            temp_score = run_model_kneighbors(s_neighbors=i+1, s_leaf=l, training=X_train, training_labels=y_train, validation_training=X_test, validation_labels=y_test)
            accuracies.append([i,l,temp_score])

0it [00:00, ?it/s]
 96%|█████████▌| 48/50 [00:02<00:00, 17.34it/s]
1it [00:02,  2.82s/it] [00:02<00:00, 17.93it/s]
 98%|█████████▊| 49/50 [00:02<00:00, 21.30it/s]
2it [00:04,  2.62s/it] [00:02<00:00, 23.35it/s]
100%|██████████| 50/50 [00:03<00:00, 12.99it/s]
3it [00:08,  2.96s/it]
 98%|█████████▊| 49/50 [00:03<00:00, 15.81it/s]
4it [00:12,  3.16s/it] [00:03<00:00, 13.88it/s]
 96%|█████████▌| 48/50 [00:02<00:00, 21.81it/s]
5it [00:15,  3.20s/it] [00:03<00:00, 15.28it/s]


In [85]:
accuracies

[[0, 5, 0.91666666666666663],
 [0, 10, 0.91666666666666663],
 [0, 15, 0.91666666666666663],
 [0, 20, 0.91666666666666663],
 [0, 25, 0.91666666666666663],
 [0, 30, 0.91666666666666663],
 [0, 35, 0.91666666666666663],
 [0, 40, 0.91666666666666663],
 [0, 45, 0.91666666666666663],
 [0, 50, 0.91666666666666663],
 [0, 55, 0.91666666666666663],
 [0, 60, 0.91666666666666663],
 [1, 5, 0.90000000000000002],
 [1, 10, 0.90000000000000002],
 [1, 15, 0.90000000000000002],
 [1, 20, 0.90000000000000002],
 [1, 25, 0.90000000000000002],
 [1, 30, 0.90000000000000002],
 [1, 35, 0.90000000000000002],
 [1, 40, 0.90000000000000002],
 [1, 45, 0.90000000000000002],
 [1, 50, 0.90000000000000002],
 [1, 55, 0.90000000000000002],
 [1, 60, 0.90000000000000002],
 [2, 5, 0.93333333333333335],
 [2, 10, 0.93333333333333335],
 [2, 15, 0.93333333333333335],
 [2, 20, 0.93333333333333335],
 [2, 25, 0.93333333333333335],
 [2, 30, 0.93333333333333335],
 [2, 35, 0.93333333333333335],
 [2, 40, 0.93333333333333335],
 [2, 45, 0.

In [86]:
final_accuracy_kneigh = []

for i in neigh_values:
    for l in leaf_values:
        count = 0
        total = 0
        for a in accuracies:
            if a[0] == i and a[1] == l:
                total = total + a[2]
                count += 1
                if count == splits_to_do:
                    total = total / splits_to_do
                    final_accuracy_kneigh.append([i, l, total])

In [87]:
final_accuracy_kneigh

[[0, 5, 0.92333333333333323],
 [0, 10, 0.91999999999999993],
 [0, 15, 0.91999999999999993],
 [0, 20, 0.91999999999999993],
 [0, 25, 0.91999999999999993],
 [0, 30, 0.91999999999999993],
 [0, 35, 0.91999999999999993],
 [0, 40, 0.91999999999999993],
 [0, 45, 0.91999999999999993],
 [0, 50, 0.91999999999999993],
 [0, 55, 0.91999999999999993],
 [0, 60, 0.91999999999999993],
 [1, 5, 0.89333333333333331],
 [1, 10, 0.89333333333333331],
 [1, 15, 0.89333333333333331],
 [1, 20, 0.89333333333333331],
 [1, 25, 0.89333333333333331],
 [1, 30, 0.89333333333333331],
 [1, 35, 0.89333333333333331],
 [1, 40, 0.89333333333333331],
 [1, 45, 0.89333333333333331],
 [1, 50, 0.89333333333333331],
 [1, 55, 0.89333333333333331],
 [1, 60, 0.89333333333333331],
 [2, 5, 0.91999999999999993],
 [2, 10, 0.91999999999999993],
 [2, 15, 0.91999999999999993],
 [2, 20, 0.91999999999999993],
 [2, 25, 0.91999999999999993],
 [2, 30, 0.91999999999999993],
 [2, 35, 0.91999999999999993],
 [2, 40, 0.91999999999999993],
 [2, 45, 0.

In [88]:
top_accuracy_kneigh = []

for f in range(len(final_accuracy_kneigh)):
    top_accuracy_kneigh.append(final_accuracy_kneigh[f][2])

In [90]:
top_accuracy_kneigh = max(top_accuracy_kneigh)

In [91]:
top_accuracy_kneigh

0.92333333333333323

In [93]:
report.append(['KNeighbors',top_accuracy_kneigh,run_model_kneighbors(training=points, training_labels=labels, validation_training=points_test, validation_labels=labels_test, s_leaf=5, s_neighbors=1)])

In [94]:
report

[['linear', 0.59000000000000008, 0.58999999999999997],
 ['poly', 0.66666666666666663, 0.67500000000000004],
 ['rbf', 0.94333333333333336, 0.95999999999999996],
 ['logistic', 0.59000000000000008, 0.58999999999999997],
 ['KNeighbors', 0.92333333333333323, 0.93500000000000005]]

In [95]:
depth_values = list(range(50))
min_samples_values = list(range(10))

In [108]:
accuracies = []
for train_index, test_index in tqdm(kf.split(X)):
    for i in tqdm(depth_values):
        for s in min_samples_values:
            temp_score = run_model_dtree(s_depth=i+1, s_split=s+2, training=X_train, training_labels=y_train, validation_training=X_test, validation_labels=y_test)
            accuracies.append([i,s,temp_score])

0it [00:00, ?it/s]
 92%|█████████▏| 46/50 [00:00<00:00, 61.22it/s]
1it [00:00,  1.39it/s] [00:00<00:00, 69.69it/s]
100%|██████████| 50/50 [00:00<00:00, 54.25it/s]
2it [00:01,  1.30it/s]
 86%|████████▌ | 43/50 [00:00<00:00, 76.42it/s]
3it [00:02,  1.38it/s] [00:00<00:00, 80.61it/s]
 94%|█████████▍| 47/50 [00:00<00:00, 56.76it/s]
4it [00:03,  1.25it/s] [00:00<00:00, 52.14it/s]
 98%|█████████▊| 49/50 [00:00<00:00, 49.92it/s]
5it [00:04,  1.19it/s] [00:00<00:00, 53.13it/s]


In [109]:
accuracies

[[0, 0, 0.48333333333333334],
 [0, 1, 0.48333333333333334],
 [0, 2, 0.48333333333333334],
 [0, 3, 0.48333333333333334],
 [0, 4, 0.48333333333333334],
 [0, 5, 0.48333333333333334],
 [0, 6, 0.48333333333333334],
 [0, 7, 0.48333333333333334],
 [0, 8, 0.48333333333333334],
 [0, 9, 0.48333333333333334],
 [1, 0, 0.59999999999999998],
 [1, 1, 0.59999999999999998],
 [1, 2, 0.59999999999999998],
 [1, 3, 0.59999999999999998],
 [1, 4, 0.59999999999999998],
 [1, 5, 0.59999999999999998],
 [1, 6, 0.59999999999999998],
 [1, 7, 0.59999999999999998],
 [1, 8, 0.59999999999999998],
 [1, 9, 0.59999999999999998],
 [2, 0, 0.73333333333333328],
 [2, 1, 0.73333333333333328],
 [2, 2, 0.73333333333333328],
 [2, 3, 0.73333333333333328],
 [2, 4, 0.73333333333333328],
 [2, 5, 0.73333333333333328],
 [2, 6, 0.73333333333333328],
 [2, 7, 0.73333333333333328],
 [2, 8, 0.73333333333333328],
 [2, 9, 0.73333333333333328],
 [3, 0, 1.0],
 [3, 1, 1.0],
 [3, 2, 1.0],
 [3, 3, 1.0],
 [3, 4, 1.0],
 [3, 5, 1.0],
 [3, 6, 1.0],
 [

In [105]:
report.append(['DecisionTreeClassifier','1.0','1.0'])

In [111]:
accuracies = []
for train_index, test_index in tqdm(kf.split(X)):
    for i in tqdm(depth_values):
        for s in min_samples_values:
            temp_score = run_model_rforrest(s_depth=i+1, s_split=s+2, training=X_train, training_labels=y_train, validation_training=X_test, validation_labels=y_test)
            accuracies.append([i,s,temp_score])

0it [00:00, ?it/s]
100%|██████████| 50/50 [00:14<00:00,  3.82it/s]
1it [00:14, 14.92s/it]
100%|██████████| 50/50 [00:13<00:00,  3.20it/s]
2it [00:28, 14.60s/it]
100%|██████████| 50/50 [00:13<00:00,  4.81it/s]
3it [00:42, 14.25s/it]
100%|██████████| 50/50 [00:09<00:00,  5.60it/s]
4it [00:51, 12.79s/it]
100%|██████████| 50/50 [00:10<00:00,  5.22it/s]
5it [01:02, 12.12s/it]


In [112]:
accuracies

[[0, 0, 0.48333333333333334],
 [0, 1, 0.48333333333333334],
 [0, 2, 0.48333333333333334],
 [0, 3, 0.48333333333333334],
 [0, 4, 0.48333333333333334],
 [0, 5, 0.48333333333333334],
 [0, 6, 0.48333333333333334],
 [0, 7, 0.48333333333333334],
 [0, 8, 0.48333333333333334],
 [0, 9, 0.48333333333333334],
 [1, 0, 0.38333333333333336],
 [1, 1, 0.41666666666666669],
 [1, 2, 0.41666666666666669],
 [1, 3, 0.5],
 [1, 4, 0.46666666666666667],
 [1, 5, 0.5],
 [1, 6, 0.41666666666666669],
 [1, 7, 0.56666666666666665],
 [1, 8, 0.46666666666666667],
 [1, 9, 0.43333333333333335],
 [2, 0, 0.48333333333333334],
 [2, 1, 0.5],
 [2, 2, 0.51666666666666672],
 [2, 3, 0.53333333333333333],
 [2, 4, 0.51666666666666672],
 [2, 5, 0.58333333333333337],
 [2, 6, 0.48333333333333334],
 [2, 7, 0.51666666666666672],
 [2, 8, 0.58333333333333337],
 [2, 9, 0.56666666666666665],
 [3, 0, 0.81666666666666665],
 [3, 1, 0.75],
 [3, 2, 0.69999999999999996],
 [3, 3, 0.71666666666666667],
 [3, 4, 0.73333333333333328],
 [3, 5, 0.733

In [113]:
final_accuracy_kneigh = []

for i in depth_values:
    for s in min_samples_values:
        count = 0
        total = 0
        for a in accuracies:
            if a[0] == i and a[1] == s:
                total = total + a[2]
                count += 1
                if count == splits_to_do:
                    total = total / splits_to_do
                    final_accuracy_kneigh.append([i, s, total])

In [114]:
final_accuracy_kneigh

[[0, 0, 0.48333333333333328],
 [0, 1, 0.48333333333333328],
 [0, 2, 0.48333333333333328],
 [0, 3, 0.48333333333333328],
 [0, 4, 0.48333333333333328],
 [0, 5, 0.48333333333333328],
 [0, 6, 0.48333333333333328],
 [0, 7, 0.48333333333333328],
 [0, 8, 0.48333333333333328],
 [0, 9, 0.48333333333333328],
 [1, 0, 0.44666666666666666],
 [1, 1, 0.45000000000000001],
 [1, 2, 0.42333333333333334],
 [1, 3, 0.5],
 [1, 4, 0.46666666666666667],
 [1, 5, 0.46666666666666667],
 [1, 6, 0.45333333333333331],
 [1, 7, 0.51000000000000001],
 [1, 8, 0.46666666666666662],
 [1, 9, 0.48333333333333339],
 [2, 0, 0.53000000000000003],
 [2, 1, 0.59333333333333338],
 [2, 2, 0.59333333333333327],
 [2, 3, 0.60333333333333328],
 [2, 4, 0.54333333333333333],
 [2, 5, 0.55000000000000004],
 [2, 6, 0.54000000000000004],
 [2, 7, 0.59999999999999998],
 [2, 8, 0.55333333333333345],
 [2, 9, 0.52333333333333332],
 [3, 0, 0.73333333333333339],
 [3, 1, 0.69000000000000006],
 [3, 2, 0.77666666666666662],
 [3, 3, 0.7233333333333332

In [115]:
top_accuracy_rforrest = []

for f in range(len(final_accuracy_kneigh)):
    top_accuracy_rforrest.append(final_accuracy_kneigh[f][2])

In [117]:
top_accuracy_rforrest = max(top_accuracy_rforrest)

In [118]:
top_accuracy_rforrest

0.97333333333333327

In [120]:
report.append(['RandomForestClassifier',top_accuracy_rforrest,run_model_rforrest(training=points, training_labels=labels, validation_training=points_test, validation_labels=labels_test, s_depth=48, s_split=2)])

In [124]:
report_df = pd.DataFrame(report)

In [126]:
report_df.to_csv('/Users/JJ/Documents/SIPA/Semester 4/Artificial Intelligence/Coding/Assignment 3/output3.csv', header = None, index = None)